# TODO
rendere disponibile mha per vedere i pesi dell'attenzione così da capire cosa sfrutta per predire il token i+1-esimo.

# Introduction
In this laboratory we will get our hands dirty working with Large Language Models (e.g. GPT and BERT) to do various useful things. I you haven't already, it is highly recommended to:

+ Read the [Attention is All you Need](https://arxiv.org/abs/1706.03762) paper, which is the basis for all transformer-based LLMs.
+ Watch (and potentially *code along*) with this [Andrej Karpathy video](https://www.youtube.com/watch?v=kCc8FmEb1nY) which shows you how to build an autoregressive GPT model from the ground up.

# Exercise 1: Warming Up
In this first exercise you will train a *small* autoregressive GPT model for character generation (the one used by Karpathy in his video) to generate text in the style of Dante Aligheri. Use [this file](https://archive.org/stream/ladivinacommedia00997gut/1ddcd09.txt), which contains the entire text of Dante's Inferno (**note**: you will have to delete some introductory text at the top of the file before training). Train the model for a few epochs, monitor the loss, and generate some text at the end of training. Qualitatively evaluate the results 

In [1]:
from torch.utils.data import Dataset
import tiktoken

class DanteSet(Dataset):
    def __init__(self, src_path, train=True):
        with open(src_path, 'r', encoding='utf-8') as f:
            self.data = f.read()
        self.enc = tiktoken.get_encoding("gpt2")
        self.t_p = 0.9
        self.enc = tiktoken.get_encoding("gpt2")
        if train:
            self.ids = self.enc.encode_ordinary(self.data[:int(len(self.data)*self.t_p)])
        else:
            self.ids = self.enc.encode_ordinary(self.data[int(len(self.data)*self.t_p):])
    def __len__(self):
        return len(self.ids)
    
    def __getitem__(self, idx):
        return self.ids[idx], self.ids[idx+1]
    
train_ds = DanteSet("inferno.txt", train=True)
test_ds = DanteSet("inferno.txt", train=False)
print(len(train_ds), len(test_ds))
print(train_ds[0])
print(train_ds.enc.decode(train_ds[0]))

71220 8005
(13534, 360)
LA D


In [10]:
# fake dataloader
import random
import torch
seq_length = 8
batch_size = 4
shuffle = True
def make_batch(batch_size, seq_length, shuffle=True, targets=True):
    batch = torch.zeros(batch_size, seq_length, dtype=torch.long)
    if targets:
        target_batch = torch.zeros(batch_size, seq_length, dtype=torch.long)
    else:
        target_batch = None
    # print(batch.shape)
    for b in range(batch_size):
        starting_index = b
        if shuffle:
            starting_index = random.randint(0, len(train_ds)-seq_length)
        for seq in range(seq_length):
            sequence = train_ds[starting_index+seq]
            batch[b][seq] = sequence[0]
            if targets:
                target_batch[b][seq] = sequence[1]
    return batch, target_batch
batch, targets = make_batch(batch_size, seq_length, shuffle, targets=True)
for b in batch:
    print(train_ds.enc.decode(b.tolist()), "\n")

for b in targets:
    print(train_ds.enc.decode(b.tolist()), "\n")

 maladetta, fredda e 

lar mi trassi.

Ed 

 si rivolse per la str 


  tal che convien che l 

adetta, fredda e gre 

 mi trassi.

Ed ecc 

 rivolse per la strada 

  tal che convien che lui 



In [11]:
import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU()
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

In [12]:
@dataclass
class GPTConfig:
    block_size: int = 16
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

config = GPTConfig()

In [39]:
import torch.nn as nn
class GPTLLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self._vocab_size = config.vocab_size
        self._n_embd = config.n_embd
        self._block_size = config.block_size
        self.token_embedding_table = nn.Embedding(self._vocab_size, self._n_embd)
        self.pos_embedding_table = nn.Embedding(self._block_size, self._n_embd)
        self.blocks = nn.Sequential(*[Block(config) for _ in range(4)])
        self.ln_f = LayerNorm(self._n_embd, bias=config.bias)
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

    
    def forward(self, x, targets = None):
        (B, T) = x.shape
        tok_emb = self.token_embedding_table(x) # (B, T, C)
        pos_emb = self.pos_embedding_table(torch.arange(T, device=x.device))
        x = tok_emb + pos_emb
        for block in self.blocks:
            x = block(x)
        x = self.ln_f(x)
        if targets is not None:
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        self.eval()
        with torch.no_grad():
            for _ in range(max_new_tokens):
                idx_cond = idx[:, -self._block_size:]
                logits, _ = self.forward(idx_cond)
                logits = logits[:, -1, :]
                probs = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probs, num_samples=1)
                idx = torch.cat((idx, idx_next), dim=-1)
        self.train()
        return idx

In [40]:
gpt = GPTLLM(config).to("cuda")
#print(gpt)
x, y = make_batch(4, config.block_size, shuffle)
x = x.to("cuda")
y = y.to("cuda")
print(x.shape, y.shape)
logits, loss = gpt(x,y)


torch.Size([4, 16]) torch.Size([4, 16])


In [47]:
from tqdm import tqdm
iterations = int(1e4)
print(f"training the model for {iterations} iterations...")
device = "cuda" if torch.cuda.is_available() else "cpu"
gpt = GPTLLM(config).to(device)
print(f"training on ", device)
optim = torch.optim.AdamW(gpt.parameters(), lr=6e-4)
for i in tqdm(range(iterations)):
    x, y = make_batch(4, config.block_size, shuffle)
    x, y = x.to(device), y.to(device)
    optim.zero_grad()
    logits, loss = gpt(x, y)
    loss.backward()
    optim.step()
    if (i + 1) % 500 == 0 : 
        print(train_ds.enc.decode(gpt.generate(torch.zeros((1,1), dtype=torch.long, device=device), max_new_tokens=config.block_size).tolist()[0]))

training the model for 10000 iterations...
training on  cuda


  5%|▌         | 508/10000 [00:08<02:36, 60.49it/s]

! me assue;
  m'io mis, in quest sta




 10%|█         | 1012/10000 [00:16<02:30, 59.88it/s]

! rirmeunara che sc,

  qual ma cantilleam


...

!
  credi da poi a seppi, Fiorenhiam


 95%|█████████▌| 9511/10000 [02:33<00:08, 58.84it/s]

!

E morta z
 ,' com' monto de la


100%|██████████| 10000/10000 [02:41<00:00, 61.81it/s]

!>>.

S' io avaro l'altissiun



In [49]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(train_ds.enc.decode(gpt.generate(context, max_new_tokens = 1024)[0].tolist()))

!>>.

Noi: <<Maestro; posdiiglio,, che' siega, fin,
  pori confitto dauglia, di scorpiese e per lo sacinto.

E io cor sia pensire in forza mia

di proceduglia avante.

<<Dolta drannia che pone pote>>.

Poi miaan lor inver'arte levar tenere sanza.

E per che dentrocciate il tristo, dostizator
  e

Non si torcenda comincaper "Tutti.


I' di quel suon mal sospesi.

Traentro, e
  portta ebbi, suo mi si` aspazzi tutti>>,
  e io; e disio,
  maestro: <<Gia ch'io nomi e Senuoi,
  o iri' son le parole conaro, vogne a mezza;
  sen giu` tristizia` d'unque;

  una si le sueelli piu` lasciando!

I' che si squanza lungo

deli e dor diere li e li oreci ch'i'armi fredi,
  gridai con 'l duca mio mio diss si governammo e chi tutta,
  guarda re, e` che solo:

"Non sanza s'io andare, e non sospetti.

Mospesi e` mai

Traova,

Ne' i pietiine,
  per appiens, che viss'io ti vittoria
  'nosollo e` fermo com' 'l mondo era amore>>,
  gridando l'amine sospdi per sbig;

  disse 'l d lodmo per lo mondoore,
  per lo

# Exercise 2: Working with Real LLMs

Our toy GPT can only take us so far. In this exercise we will see how to use the [Hugging Face](https://huggingface.co/) model and dataset ecosystem to access a *huge* variety of pre-trained transformer models.

## Exercise 2.1: Installation and text tokenization

First things first, we need to install the [Hugging Face transformer library](https://huggingface.co/docs/transformers/index):

    conda install -c huggingface -c conda-forge transformers
    
The key classes that you will work with are `GPT2Tokenizer` to encode text into sub-word tokens, and the `GPT2LMHeadModel`. **Note** the `LMHead` part of the class name -- this is the version of the GPT2 architecture that has the text prediction heads attached to the final hidden layer representations (i.e. what we need to **generate** text). 

Instantiate the `GPT2Tokenizer` and experiment with encoding text into integer tokens. Compare the length of input with the encoded sequence length.

**Tip**: Pass the `return_tensors='pt'` argument to the togenizer to get Pytorch tensors as output (instead of lists).

In [61]:
print("Siccome transformers non mi funziona con il notebook, il codice di questa parte si trova il Lab2-LLMs")

Siccome transformers non mi funziona con il notebook, il codice di questa parte si trova il Lab2-LLMs


## Exercise 2.2: Generating Text

There are a lot of ways we can, given a *prompt* in input, sample text from a GPT2 model. Instantiate a pre-trained `GPT2LMHeadModel` and use the [`generate()`](https://huggingface.co/docs/transformers/v4.27.2/en/main_classes/text_generation#transformers.GenerationMixin.generate) method to generate text from a prompt.

**Note**: The default inference mode for GPT2 is *greedy* which might not results in satisfying generated text. Look at the `do_sample` and `temperature` parameters.

In [20]:
# Your code here.

# Exercise 3: Reusing Pre-trained LLMs (choose one)

Choose **one** of the following exercises (well, *at least* one). In each of these you are asked to adapt a pre-trained LLM (`GPT2Model` or `DistillBERT` are two good choices) to a new Natural Language Understanding task. A few comments:

+ Since GPT2 is a *autoregressive* model, there is no latent space aggregation at the last transformer layer (you get the same number of tokens out that you give in input). To use a pre-trained model for a classification or retrieval task, you should aggregate these tokens somehow (or opportunistically select *one* to use).

+ BERT models (including DistillBERT) have a special [CLS] token prepended to each latent representation in output from a self-attention block. You can directly use this as a representation for classification (or retrieval).

+ The first *two* exercises below can probably be done *without* any fine-tuning -- that is, just training a shallow MLP to classify or represent with the appropriate loss function.

# Exercise 3.1: Training a Text Classifier (easy)

Peruse the [text classification datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=downloads). Choose a *moderately* sized dataset and use a LLM to train a classifier to solve the problem.

**Note**: A good first baseline for this problem is certainly to use an LLM *exclusively* as a feature extractor and then train a shallow model.

# Exercise 3.2: Training a Question Answering Model (harder)

Peruse the [multiple choice question answering datasets on Hugging Face](https://huggingface.co/datasets?task_categories=task_categories:multiple-choice&sort=downloads). Chose a *moderately* sized one and train a model to answer contextualized multiple-choice questions. You *might* be able to avoid fine-tuning by training a simple model to *rank* the multiple choices (see margin ranking loss in Pytorch).

# Exercise 3.3: Training a Retrieval Model (hardest)

The Hugging Face dataset repository contains a large number of ["text retrieval" problems](https://huggingface.co/datasets?task_categories=task_categories:text-retrieval&p=1&sort=downloads). These tasks generally require that the model measure *similarity* between text in some metric space -- naively, just a cosine similarity between [CLS] tokens can get you pretty far. Find an interesting retrieval problem and train a model (starting from a pre-trained LLM of course) to solve it.

**Tip**: Sometimes identifying the *retrieval* problems in these datasets can be half the challenge. [This dataset](https://huggingface.co/datasets/BeIR/scifact) might be a good starting point.

si può scaricare un modello open source per fare zero shot contest question answering (non usare chatgpt3)